In [3]:
#!/usr/bin/env python3
import wmfdata as wmf
import pandas as pd
import matplotlib

matplotlib.style.use('ggplot')

spark = wmf.spark.get_custom_session(
    master="yarn",
    spark_config={
        "spark.driver.memory": "16g",
        "spark.dynamicAllocation.maxExecutors": 128,
        "spark.executor.memory": "16g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 512
    }
)

You are using wmfdata v1.3.2, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


PYSPARK_PYTHON=/usr/lib/anaconda-wmf/bin/python3


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark2/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/23 21:09:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/08/23 21:09:44 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/08/23 21:09:44 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/08/23 21:09:44 

In [4]:
LANGUAGE_EDITIONS = "('en{0}','es{0}','ja{0}','ar{0}','de{0}','pt{0}','vi{0}','fr{0}','ca{0}','ru{0}','id{0}','bn{0}','hi{0}','cs{0}')"
WIKI_FORMAT = LANGUAGE_EDITIONS.format('wiki')
PROJECT_FORMAT = LANGUAGE_EDITIONS.format('.wikipedia')

query = """
WITH pageviews AS (
SELECT 
    month, 
    year, 
    concat(substr(project,1,2),'wiki') as wiki_db,
    COUNT(*) AS num_pageviews
FROM wmf.pageview_hourly
WHERE agent_type = 'user'
AND project in {0}
GROUP BY
    month,
    year,
    project
), 

new_accounts AS (
SELECT
    cast(substr(user_first_edit_timestamp,1,4) as int) as year, 
    cast(substr(user_first_edit_timestamp,6,2) as int) as month,
    wiki_db,
    COUNT(*) AS num_new_accounts
FROM wmf.mediawiki_user_history 
WHERE snapshot = '2022-07'
AND wiki_db in {1}
AND array_contains(user_groups,'bot') = FALSE
AND user_first_edit_timestamp IS NOT NULL
GROUP BY
    cast(substr(user_first_edit_timestamp,1,4) as int),
    cast(substr(user_first_edit_timestamp,6,2) as int),
    wiki_db
),
    
articles_created AS (
SELECT
    cast(substr(start_timestamp ,1,4) as int) as year, 
    cast(substr(start_timestamp,6,2) as int) as month,
    wiki_db,
    COUNT(*) AS num_articles_created
FROM wmf.mediawiki_page_history 
WHERE snapshot = '2022-07'
AND page_namespace = 1
AND wiki_db in {1}
GROUP BY
    cast(substr(start_timestamp,1,4) as int),
    cast(substr(start_timestamp,6,2) as int),
    wiki_db
),

articles_deleted AS (
SELECT
    cast(substr(end_timestamp ,1,4) as int) as year, 
    cast(substr(end_timestamp,6,2) as int) as month,
    wiki_db,
    COUNT(*) AS num_articles_deleted
FROM wmf.mediawiki_page_history 
WHERE snapshot = '2022-07'
AND page_namespace = 1
AND wiki_db in {1}
GROUP BY
    cast(substr(end_timestamp,1,4) as int),
    cast(substr(end_timestamp,6,2) as int),
    wiki_db
),

num_articles AS (
SELECT
    articles_created.month,
    articles_created.year,
    articles_created.wiki_db,
    COALESCE(articles_deleted.num_articles_deleted,0) AS articles_deleted,
    articles_created.num_articles_created,
    articles_created.num_articles_created-COALESCE(articles_deleted.num_articles_deleted,0) AS article_diff,
    SUM(articles_created.num_articles_created-COALESCE(articles_deleted.num_articles_deleted,0)) OVER (PARTITION BY articles_created.wiki_db ORDER BY articles_created.year, articles_created.month) AS num_articles 
FROM articles_created
LEFT JOIN articles_deleted
ON (articles_created.month = articles_deleted.month AND articles_created.year = articles_deleted.year AND articles_created.wiki_db = articles_deleted.wiki_db)
WHERE articles_created.month IS NOT NULL
AND articles_created.year IS NOT NULL
)

SELECT pageviews.month, pageviews.year, pageviews.wiki_db, pageviews.num_pageviews, new_accounts.num_new_accounts, num_articles.num_articles
FROM pageviews
INNER JOIN new_accounts
ON pageviews.year = new_accounts.year AND pageviews.month = new_accounts.month AND pageviews.wiki_db = new_accounts.wiki_db
INNER JOIN num_articles
ON pageviews.year = num_articles.year AND pageviews.month = num_articles.month AND pageviews.wiki_db = num_articles.wiki_db
""".format(
    PROJECT_FORMAT,
    WIKI_FORMAT
)

df = wmf.spark.run(query)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
[Stage 0:>                                                   (32 + 116) / 10000]22/08/23 21:11:43 WARN NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@68e6a733.
[Stage 0:>                                                   (83 + 189) / 10000]22/08/23 21:11:45 WARN NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@7cbf5769.
[Stage 0:=>                                                 (205 + 429) / 10000]22/08/23 21:11:48 WARN NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@7368dbb1.
22/08/23 21:13:52 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileC

,month,year,wiki_db,num_pageviews,num_new_accounts,num_articles
0,8,2015,cswiki,37385145,681,46368
1,11,2018,bnwiki,12026938,677,72838
2,3,2019,enwiki,6489579704,69727,8448018
3,8,2019,ruwiki,620792298,4857,939735
4,3,2016,viwiki,39943008,1032,1136195
...,...,...,...,...,...,...
1225,1,2021,ruwiki,758226972,6023,1013342
1226,6,2017,bnwiki,6087056,263,61545
1227,7,2017,eswiki,804469040,8199,501030
1228,12,2019,arwiki,176104568,3906,1064872


In [9]:
FILEPATH = '/home/jmads/datasets/momentum/language_edition_first_edit_timestamps_8-23-22.csv'
min_age = pd.read_csv(FILEPATH)

def getWikiAge(df,min_age_df):
    min_age_df = min_age_df.rename(columns={'year':'year_1','month':'month_1'})
    df = df.merge(min_age_df, on='wiki_db')
    df['wiki_age'] = df['year'].multiply(12).add(df['month']).subtract(df['year_1'].multiply(12).add(df['month_1']))
    return df

result_df = df.copy()
result_df = getWikiAge(result_df,min_age)
result_df = result_df.sort_values(['wiki_db','year','month'])

In [11]:
FILEPATH = '/home/jmads/datasets/momentum/pageview_new_accounts_multi-wiki_8-24-22.csv'

result_df.to_csv(FILEPATH,index=False)

In [10]:
result_df

,month,year,wiki_db,num_pageviews,num_new_accounts,num_articles,year_1,month_1,wiki_age
653,5,2015,arwiki,99512972,3125,66698,2003,7,142
628,6,2015,arwiki,91875676,3041,67047,2003,7,143
632,7,2015,arwiki,96692677,3124,67407,2003,7,144
660,8,2015,arwiki,94689390,2765,67982,2003,7,145
667,9,2015,arwiki,93941035,2534,68326,2003,7,146
...,...,...,...,...,...,...,...,...,...
355,4,2022,viwiki,50054369,1007,1274814,2002,11,233
410,5,2022,viwiki,47176896,1071,1275505,2002,11,234
421,6,2022,viwiki,46589502,1037,1276093,2002,11,235
406,7,2022,viwiki,47359805,920,1276788,2002,11,236
